<a href="https://colab.research.google.com/github/chefPony/genai-lab/blob/master/Text_Generation_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install datasets
%pip install transformers
%pip install evaluate
%pip install genaibook
%pip install --upgrade numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.3.3 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.3 which is incompatible.
tensorflow 2.19.0 requires n

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("fancyzhx/ag_news")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [2]:
raw_datasets.column_names

{'train': ['text', 'label'], 'test': ['text', 'label']}

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from genaibook.core import get_device

device = get_device()

model_id = "HuggingFaceTB/SmolLM-135M"
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

def tokenize(batch): return tokenizer(batch["text"], truncation=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [4]:
tokenized_datasets = raw_datasets.map(tokenize, batched=True, remove_columns=["text", "label"])

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [6]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

samples = [tokenized_datasets["train"][i] for i in range(3)]

In [7]:
for s in samples:
   print(f" Sample length {len(s['input_ids'])}")

 Sample length 39
 Sample length 56
 Sample length 51


In [9]:
collated = collator(samples)
for key in collated:
   print(f"Shape {collated[key].shape}")

Shape torch.Size([3, 56])
Shape torch.Size([3, 56])
Shape torch.Size([3, 56])


## Full weights Fine Tuning

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "SmolLM-finetune-example",
    push_to_hub=False,
    eval_strategy="steps",
    per_device_train_batch_size=8,
    weight_decay=0.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    num_train_epochs=2,
    eval_steps=200,
    logging_steps=200,
    # need to add this arg otherwise it will try to access wand.ai
    report_to="none"
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(5000)),
    eval_dataset=tokenized_datasets["test"],
    data_collator=collator
)

/tmp/ipython-input-2090376721.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
200,3.104300,3.913375
400,2.939800,3.852843
600,2.805000,3.766606
800,1.675700,4.046868
1000,1.474700,4.017645
1200,1.412100,4.022408


TrainOutput(global_step=1250, training_loss=2.2036139526367187, metrics={'train_runtime': 825.8341, 'train_samples_per_second': 12.109, 'train_steps_per_second': 1.514, 'total_flos': 609392032017408.0, 'train_loss': 2.2036139526367187, 'epoch': 2.0})

In [ ]:
from transformers import pipeline

pipe = pipeline(task="text-generation", model=trainer.model,
                tokenizer=tokenizer)


Device set to use cuda:0


{'generated_text': 'Q1: Google IPO Set to Begin Today (NewsFactor) NewsFactor - Google Inc. (GOOG.O) has set its initial public offering'}

In [ ]:
print(pipe("Q1", do_sample=True, temperature=0.1, max_new_tokens=30)[0])
print(pipe("China", do_sample=True, temperature=0.1, max_new_tokens=30)[0])
print(pipe("Ocean", do_sample=True, temperature=0.1, max_new_tokens=30)[0])

{'generated_text': 'Q1: Google IPO Set to Begin Today (NewsFactor) NewsFactor - Google Inc. (GOOG.O) has set its initial public offering'}
{'generated_text': "China's Peirsol Wins 200m Freestyle Gold (AP) AP - Ailing American swimmer Michael Phelps won his second gold medal"}
{'generated_text': "Ocean to Watch on the Rise (AP) AP - Investors in the world's biggest Internet company are to get a head start on the rise of its"}


## Parameter Efficient Fine Tuning (PEFT)

In [12]:
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM-135M")
peft_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.05, task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 460,800 || all params: 134,975,808 || trainable%: 0.3414


In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "SmolLM-peft-example",
    push_to_hub=False,
    eval_strategy="steps",
    per_device_train_batch_size=8,
    weight_decay=0.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    num_train_epochs=2,
    eval_steps=200,
    logging_steps=200,
    # need to add this arg otherwise it will try to access wand.ai
    report_to="none"
)

trainer = Trainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(5000)),
    eval_dataset=tokenized_datasets["test"],
    data_collator=collator
)

/tmp/ipython-input-3240067455.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Step,Training Loss,Validation Loss
200,3.589800,3.634365
400,3.397600,3.577814
600,3.364400,3.550251
800,3.271400,3.539914
1000,3.267800,3.530550
1200,3.255000,3.529412


TrainOutput(global_step=1250, training_loss=3.353168835449219, metrics={'train_runtime': 708.0556, 'train_samples_per_second': 14.123, 'train_steps_per_second': 1.765, 'total_flos': 612036087671808.0, 'train_loss': 3.353168835449219, 'epoch': 2.0})

In [19]:
from transformers import pipeline

peft_pipe = pipeline(task="text-generation", model=trainer.model,
                tokenizer=tokenizer)

print(peft_pipe("Q1", do_sample=True, temperature=0.1, max_new_tokens=30)[0])
print(peft_pipe("China", do_sample=True, temperature=0.1, max_new_tokens=30)[0])
print(peft_pipe("Ocean", do_sample=True, temperature=0.1, max_new_tokens=30)[0])

Device set to use cuda:0


{'generated_text': 'Q1: The 100-meter dash (AP) AP - The 100-meter dash is the fastest-ever time-trial'}
{'generated_text': "China, China to Trade 100 Million in New Bills (Reuters) Reuters - China, the world's second-largest economy, has"}
{'generated_text': "Ocean's Greatest Surprise (AP) AP - The world's biggest ocean has been discovered by a team of scientists in the middle of the Pacific Ocean"}
